# importing libraries

In [1]:
import math
from math import pi

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# create sub-circuit for entanglement swapping

In [5]:
sub_q = QuantumRegister(3)
sub_circ = QuantumCircuit(sub_q, name='sub_circ')

sub_circ.cx(sub_q[0], sub_q[1])
sub_circ.h(sub_q[0])
sub_circ.cx(sub_q[1], sub_q[2])
sub_circ.cz(sub_q[0], sub_q[2])

ent_swap = sub_circ.to_instruction()

# preparing circuit

In [ ]:
# preparing registers

pair1 = QuantumRegister(2, "pair1")
pair2 = QuantumRegister(2, "pair2")
pair3 = QuantumRegister(2, "pair3")

check2 = ClassicalRegister(2, "check2")
check3 = ClassicalRegister(2, "check3")

result = ClassicalRegister(2, "result")

In [ ]:
# constructing circuit

sys = QuantumCircuit(pair1, pair2, pair3, 
                     check2, check3, 
                     result)

In [ ]:
#creating bell-pairs
sys.h(pair1[0])
sys.cx(pair1[0], pair1[1])

sys.h(pair2[0])
sys.cx(pair2[0], pair2[1])

sys.h(pair3[0])
sys.cx(pair3[0], pair3[1])

sys.barrier()


In [ ]:
#Deutsch's Correction 

#rotation-Alice
sys.rx(pi/2, pair1[0])
sys.rx(pi/2, pair2[0])
sys.rx(pi/2, pair3[0])

#rotation-Bob
sys.rx(-pi/2, pair1[1])
sys.rx(-pi/2, pair2[1])
sys.rx(-pi/2, pair3[1])

sys.barrier()

In [ ]:
#entanglement pumping
sys.cx(pair1[0], pair2[0])
sys.cx(pair1[0], pair3[0])
sys.barrier()

sys.cx(pair1[1], pair2[1])
sys.cx(pair1[1], pair3[1])
sys.barrier()

In [ ]:
#checking fidelity through bell-measurement
sys.cx(pair1[0], pair1[1])
sys.h(pair1[0])
sys.barrier()

In [ ]:
#distillation

sys.measure(pair2, check2)
sys.measure(pair3, check3)
sys.barrier()

In [ ]:
sys.measure(pair1, result)

In [ ]:
style = {'fold':20}
sys.draw(output='mpl', style=style)

# running on real device

In [ ]:
from qiskit import IBMQ

IBMQ.load_account() # Load account from disk
provider = IBMQ.get_provider(group='open')
provider.backends()

In [ ]:
backend = provider.get_backend('ibmq_16_melbourne')

In [ ]:
from qiskit import execute
from qiskit.tools.monitor import job_monitor

In [ ]:
shots = 1024
job = execute(sys, backend=backend, shots=shots)

print(job.job_id())
job_monitor(job)

In [ ]:
job.error_message()

# measurement error mitigation

In [ ]:
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)

In [ ]:
cal_circuits, state_labels = complete_meas_cal(qr=sys.qregs[0], 
                                               circlabel='measurement_calibration')

In [ ]:
len(cal_circuits)